# Pandas functions useful for finding formatting and sanitizing problems

In order to check your data for formatting problems, you will probably need to use some string operations. Some of these are demonstrated in the remaining of thi notebook. For more examples, check ["Working with text data"](https://pandas.pydata.org/docs/user_guide/text.html) on Pandas docs.


# Imports

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


C:\Users\richa\AppData\Local\Temp\ipykernel_23548\2122269801.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Telilng pandas to diplay more dataframe content

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 51)
pd.set_option('display.width', 1000)

# (Down)Load data

In [7]:


file_path = r"\Users\richa\Documents\EPF\MDE\DataCleaning\data.csv"
data = pd.read_csv(file_path)

FileNotFoundError: [Errno 2] No such file or directory: '\\Users\\richa\\Documents\\EPF\\MDE\\DataCleaning\\data.csv'

# Examples

## Checking all values in a column

In [4]:
data.adr_num.values

array([' ', ' ', ' ', ' ', '155', '154', '1', '3490', ' ', '20', '830',
       ' ', '135', '21', '280', '570', '14', '164', '41', '3', '28', ' ',
       '50', ' ', '10', '842', '2', '950', '14', '1', '2', ' ', '280',
       '6', '125', '655', '45', '158', '119', '14', '-', '1000', '85',
       '99', '13', '43', '170', '-', '424 - 460', '843', '196 - 156',
       '99', '300', '123', '110', '20', '230', '237', '67', '1933', '551',
       '971', '380', '100', '122', '26', '215', '-', '60', '-', ' ', '88',
       '1375', '183', '219 - 289', '45', '5', '117', '50', '8', '1',
       '19 bis', '1945', '789', '25', ' ', '1', ' ', '150', ' ', '18',
       '1', '237', '1071', '118', '111', '55', '280', ' ', ' ', '-', '-',
       '-', ' ', '50', '50', '2', '67', '419', '1184', ' ', '260', ' ',
       ' ', ' ', '175', ' ', ' ', '10', '13', '3', '1', '1247', '64', '7',
       '205', '1330', '74', '501', '-', '694 -700', '240', ' ', ' ', '16',
       '2', ' ', '1784', '50', '50', '50', '130', '18', 

## Checking for unique values

In [5]:
data.com_nom.unique()

array(['Montpellier', ' ', 'MONTPELLIER'], dtype=object)

## Split a string into columns
A string can be split into its composing parts, with each going to a particular column. Here is an example:

In [6]:
split_phones = data.tel1.str.split(expand=True)
split_phones

,0,1,2,3,4
0,334,67,64,87,70
1,None,None,None,None,None
2,None,None,None,None,None
3,None,None,None,None,None
4,+334,67,40,33,57
...,...,...,...,...,...
270,334,67,54,45,20
271,334,67,75,44,43
272,334,67,34,71,33
273,None,None,None,None,None


The fact that we got 5 columns out of the split means that at most phones are writen in 5 space-separated parts.


### Using counting and masking to select atypical cases

 At this point one thing you can do is count the non empty elements across each row, and use that info to see what are the values not matching the this 5-group pattern:

In [7]:
not_five = split_phones.count(axis=1) != 5
not_zero = split_phones.count(axis=1) != 0
data.tel1.loc[not_five & not_zero]

45    -
69    -
70    -
Name: tel1, dtype: object

### Combine with unique and find possible phone preffixes

You can also split at only the first whitespace and check the first column to see the different ways phone preffixes got written.

In [8]:
data.tel1.str.split(n=1, expand=True)

,0,1
0,334,67 64 87 70
1,None,None
2,None,None
3,None,None
4,+334,67 40 33 57
...,...,...
270,334,67 54 45 20
271,334,67 75 44 43
272,334,67 34 71 33
273,None,None


In [9]:
data.tel1.str.split(n=1, expand=True)[0].unique()

array(['334', None, '+334', '-', '336', '06', '337'], dtype=object)

## Identify different formats in strings

You can use [regular expressions](https://www.geeksforgeeks.org/python-regex-cheat-sheet/) to check if strings match a particular pattern. 


For instance, to check for phone numbers following the pattern:
- 33x xx xx xx xx --> use  `r"33\d \d\d \d\d \d\d \d\d"`
- +33x xx xx xx xx--> use `r"\+33\d \d\d \d\d \d\d \d\d"`
- +33 0x xx xx xx xx--> use `r"\+33 0\d \d\d \d\d \d\d \d\d"`
- either of the precedent --> use `r"\+*33 *0*\d \d\d \d\d \d\d \d\d"`

If you want to check if a string starts with the pattern , use `match`. If you want a strict check where all string is a match, use `full match`. Here are some usage examples.

If you are not familiar with regular expressions, I suggest you take some time to go through a tutorial, like ["Regular expressions in Python" from Google for Education](https://developers.google.com/edu/python/regular-expressions).

If you just need a quick syntax refresher, a [cheatsheet](https://www.geeksforgeeks.org/python-regex-cheat-sheet/) should suffice. If you want to easily play around with your regex to validate it, you can use one of the online regex testers like [regex101](https://regex101.com/)


### String matches 0+ whitespace characters

In [10]:
only_whitespace = data.tel1.str.fullmatch(r"\s*")

print("Count of strings matching the pattern")
data.tel1[only_whitespace].count()

Count of strings matching the pattern


49

### String matches +33x xx xx xx xx or 33x xx xx xx xx

In [11]:
follows_pattern = data.tel1.str.fullmatch(
    pat = r'\+*33\d \d\d \d\d \d\d \d\d'
)

print ("Count of strings NOT matching the pattern: ")
data.tel1[~follows_pattern].count()

Count of strings NOT matching the pattern: 


69

### Dates not matching XXXX-XX-XX pattern

In [12]:
date_fmt = data.dermnt.str.match(r"\d\d\d\d-\d\d-\d\d")
data.dermnt[~date_fmt].unique()

array([' ', '2018-12-6', 'Tous les ans'], dtype=object)